# Contextual Precision

Contextual Precision ใช้สำหรับการประเมิน RAG โดยประเมินจาก retrieval_context ว่ามีความเกี่ยวข้องกับ input ไหม 

### Required Arguments
- input
- actual_output
- expected_output
- retrieval_context

In [1]:
from langchain_openai import AzureChatOpenAI
from deepeval.models.base_model import DeepEvalBaseLLM
import sys
sys.path.append('/opt/project/src/evaluate_llm/')
from api_key_config import settings
import os

os.environ["OPENAI_API_VERSION"] = settings.OPENAI_API_VERSION
os.environ["OPENAI_API_KEY"] = settings.OPENAI_API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = settings.AZURE_OPENAI_ENDPOINT

class AzureOpenAI(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Custom Azure OpenAI Model"

# Replace these with real values
custom_model = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
)
azure_openai = AzureOpenAI(model=custom_model)

/usr/local/lib/python3.9/site-packages/deepeval/__init__.py:42: UserWarning: You are using deepeval version 0.21.42, however version 0.21.45 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [2]:
from deepeval import evaluate
from deepeval.metrics import ContextualPrecisionMetric
from deepeval.test_case import LLMTestCase

# Replace this with the actual output from your LLM application
actual_output = "We offer a 30-day full refund at no extra cost."

# Replace this with the expected output from your RAG generator
expected_output = "You are eligible for a 30 day full refund at no extra cost."

# Replace this with the actual retrieved context from your RAG pipeline
retrieval_context = ["All customers are eligible for a 30 day full refund at no extra cost."]

metric = ContextualPrecisionMetric(
    threshold=0.7,
    model=azure_openai,
    include_reason=True
)
test_case = LLMTestCase(
    input="What if these shoes don't fit?",
    actual_output=actual_output,
    expected_output=expected_output,
    retrieval_context=retrieval_context
)

metric.measure(test_case)
print(metric.score)
print(metric.reason)

# or evaluate test cases in bulk
evaluate([test_case], [metric])

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

0
The score is 0.00 because the only retrieval context provided has a 'no' verdict and the reason for the 'no' verdict is that "The context does not address the question about the shoes not fitting, it only mentions the refund policy." This shows that the relevant nodes are not ranked higher than irrelevant nodes, resulting in a low contextual precision score.
Evaluating test cases...
Event loop is already running. Applying nest_asyncio patch to allow async execution...




Metrics Summary

  - ❌ Contextual Precision (score: 0, threshold: 0.7, strict: False, evaluation model: Custom Azure OpenAI Model, reason: The score is 0.00 because the only node in the retrieval context received a 'no' verdict and was ranked as the only node. Therefore, there were no relevant nodes ranked higher than irrelevant nodes, resulting in a low contextual precision score., error: None)

For test case:

  - input: What if these shoes don't fit?
  - actual output: We offer a 30-day full refund at no extra cost.
  - expected output: You are eligible for a 30 day full refund at no extra cost.
  - context: None
  - retrieval context: ['All customers are eligible for a 30 day full refund at no extra cost.']


Overall Metric Pass Rates

ContextualPrecisionMetric: 0.00% pass rate




/usr/local/lib/python3.9/site-packages/portalocker/utils.py:218: UserWarning: timeout has no effect in blocking mode
  warnings.warn(


✅ Tests finished! Run "deepeval login" to view evaluation results on the web.

[TestResult(success=False, metrics=[<deepeval.metrics.contextual_precision.contextual_precision.ContextualPrecisionMetric object at 0x7fbc046d66a0>], input="What if these shoes don't fit?", actual_output='We offer a 30-day full refund at no extra cost.', expected_output='You are eligible for a 30 day full refund at no extra cost.', context=None, retrieval_context=['All customers are eligible for a 30 day full refund at no extra cost.'])]

There are five optional parameters when creating a ContextualPrecisionMetric:

- [Optional] threshold: a float representing the minimum passing threshold, defaulted to 0.5.
- [Optional] model: a string specifying which of OpenAI's GPT models to use, OR any custom LLM model of type DeepEvalBaseLLM. Defaulted to 'gpt-4o'.
- [Optional] include_reason: a boolean which when set to True, will include a reason for its evaluation score. Defaulted to True.
- [Optional] strict_mode: a boolean which when set to True, enforces a binary metric score: 1 for perfection, 0 otherwise. It also overrides the current threshold and sets it to 1. Defaulted to False.
- [Optional] async_mode: a boolean which when set to True, enables concurrent execution within the measure() method. Defaulted to True.

### How Is It Calculated?

![alt text](contextual_precision_formular.png)

ข้อมูลเพิ่มเติม:
- \(k\) คือโหนดลำดับที่ (i+1) ใน retrieval_context
- \(n\) คือความยาวของ retrieval_context
- \(r_k\) คือค่า relevance แบบไบนารีสำหรับโหนดที่ \(k\) ใน retrieval_context ซึ่ง \(r_k = 1\) สำหรับโหนดที่เกี่ยวข้อง และ \(0\) สำหรับโหนดที่ไม่เกี่ยวข้อง

ContextualPrecisionMetric ใช้ LLM เพื่อกำหนดว่าโหนดแต่ละตัวใน retrieval_context นั้นเกี่ยวข้องกับ input หรือไม่ โดยพิจารณาข้อมูลจาก expected_output จากนั้นจะคำนวณ weighted cumulative precision เป็นคะแนนความแม่นยำเชิงบริบท

Weighted cumulative precision (WCP) ถูกใช้เพราะว่า:
- เน้นผลลัพธ์ที่อยู่ลำดับต้นๆ: WCP ให้ความสำคัญกับความเกี่ยวข้องของผลลัพธ์ที่ถูกจัดอันดับต้นๆ มากกว่า ซึ่งสำคัญเนื่องจาก LLM มักให้ความสำคัญกับโหนดที่อยู่ลำดับต้นๆ ใน retrieval_context (ซึ่งอาจทำให้เกิดการแสดงผลที่ผิดพลาดถ้าโหนดถูกจัดอันดับไม่ถูกต้อง)
- ให้รางวัลการจัดอันดับที่เกี่ยวข้อง: WCP สามารถจัดการกับระดับความเกี่ยวข้องที่แตกต่างกันได้ (เช่น "เกี่ยวข้องมาก", "ค่อนข้างเกี่ยวข้อง", "ไม่เกี่ยวข้อง") ซึ่งแตกต่างจากเมตริกเช่น precision ที่ถือว่าโหนดที่ถูกดึงกลับทั้งหมดมีความสำคัญเท่าเทียมกัน

คะแนนความแม่นยำเชิงบริบทที่สูงกว่าหมายถึงความสามารถที่มากขึ้นของระบบการดึงข้อมูลในการจัดอันดับโหนดที่เกี่ยวข้องสูงขึ้นใน retrieval_context